# Arabic Poems

### Introduction

Poetry is so influential because it helps us understand the culture of world around us and appreciate. Any language has poetry as part of its culture like Arabic.<br>

Arabic is the official language of the 22 countries that form the Arab League. There are more than 400 million Arabic speakers across the world, though they predominantly live in the region stretching across the Middle East and North Africa. [1]



### The problem

We need a model that can predict the poet's name for a poem



### Submission Format

Submission files should contain two columns: poem_id and poet_id. `<br>

The file should contain a header and have the following format:<br>
> poem_id,poet_id<br>
> 166,14<br>



### Swag prizes

> Swag prizes for top winners are sponsored by Google Developers. The number of <br>
> team members are limited to 3.<br>
> Share your progress using this hashtag on Twitter : #MLOlympiad<br>



### There are some restrictions.
- Google doesn't support competitions in Crimea, North Korea, Quebec, and where prohibited by law. <br>
This program follows Google Coding Competitions Terms, for more details see:<br>
https://codingcompetitions.withgoogle.com/terms .
- Swag deliveries can be delayed in some regions due to COVID-19.

References:
[1] https://www.britishcouncil.org/voices-magazine/surprising-facts-about-arabic-language

[ML Olympiad](https://www.kaggle.com/c/ml-olympiad-arabic-poems)

## Implementing process

To Implement NLP Project You Need To Do The Following:
- **Normalize**, such as removing any text from signs (,][;'/) and any thing that might effict the text
- **Tokenize**, split each word Individual to be as <"aziz", "is","a","nice","person"> 
- **Remove Stop Words**, removing any word that will be usfult such as <"aziz","nice","person"> to reduce the voulum of your text
- **Stem or Lemmatize**, To Simplify it, **Stemming** is the process of reducing a word to its stem or root form. For example, branching, branched, and branches all stem from the word branch. and **Lemmatization** is the process to map the words back to its root using a dictionary. For example, is, was, and were would all be lemmatized to "be".

To summary:


|   Text Processing |	Example  	|  
|:-:	|---	|
|   Given |	Jenna went back to University   |
|  Normalized |	jenna went back to university |
| Tokenized |	<"jenna", "went", "back", "to", "University">   |
|  Stop Word Removal |	<"jenna", "went", "university"> |
| Stem & Lemmatized |	<"jenna", "go", "univers"> |


### Feature Extraction

To convert the way is represnted from text to numbercal way so that the computer could understand it, some way are the most common ways which are:
- Bags of Words
- Term Frequency - Inverse Document Frequency (TF-IDF) Transform 
- One-Hot Encoding
- Word Embeddings
- Word2Vec

# Importing

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns  # for data Visualisation
import matplotlib.pyplot as plt # for data Visualisation
import re # for preprocessing text
import string # for preprocessing text


%matplotlib inline

from pandas_profiling import ProfileReport # for Simplify EDA

import nltk # for processing texts
from nltk.corpus import stopwords # list of stop words
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



import pyarabic.araby as araby # provides basic functions to manipulate Arabic letters and text

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdulazizalkhonain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Reading files 
This process contains redain the files and check the companination of them

In [76]:
# Sample submission data
sample_submission_df = pd.read_csv('Datasets/sample_submission.csv')
# poet name ID data
poet_nameID_df = pd.read_csv('Datasets/poet_nameid.csv')
# test and train data
train_df = pd.read_csv('Datasets/train.csv')
test_df = pd.read_csv('Datasets/test.csv')


In [100]:
print(sample_submission_df.info())
print('\n\nThe Shape is {}\n\n'.format(sample_submission_df.shape))
sample_submission_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   poem_id  217 non-null    int64
 1   poet_id  217 non-null    int64
dtypes: int64(2)
memory usage: 3.5 KB
None


The Shape is (217, 2)




,poem_id,poet_id
0,28111,44
1,28112,50
2,28113,56
3,28114,62
4,28115,68


In [101]:
print(poet_nameID_df.info())
print('\n\nThe Shape is {}\n\n'.format(poet_nameID_df.shape))
poet_nameID_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   poet_id    652 non-null    int64 
 1   poet_name  652 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.3+ KB
None


The Shape is (652, 2)




,poet_id,poet_name
0,2,بدر شاكر السياب
1,7,نزار قباني
2,9,أمل دنقل
3,10,فاروق جويدة
4,13,أولاد أحمد


In [102]:
print(train_df.info())
print('\n\nThe Shape is {}\n\n'.format(train_df.shape))
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57812 entries, 0 to 57811
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   poem_id     57812 non-null  int64 
 1   poem_link   57812 non-null  object
 2   poem_style  57812 non-null  object
 3   poem_text   57809 non-null  object
 4   poem_title  57810 non-null  object
 5   poet_cat    57812 non-null  object
 6   poet_id     57812 non-null  int64 
 7   poet_link   57812 non-null  object
 8   poet_name   57812 non-null  object
dtypes: int64(2), object(7)
memory usage: 4.0+ MB
None


The Shape is (57812, 9)




,poem_id,poem_link,poem_style,poem_text,poem_title,poet_cat,poet_id,poet_link,poet_name
0,21,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,أنشودة المطر,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
1,65546,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,أقداح و أحلام,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
2,65561,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة و...,هوى واحد !,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
3,66170,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,لا تزيديه لوعة فهو يلقاك لينسي لديك بعض اكتءا...,لا تزيديه لوعة,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
4,66171,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,عطرت احلامي بهذا الشذي من شعرك المسترسل الاسو...,عبير,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب


In [103]:
print(test_df.info())
print('\n\nThe Shape is {}\n\n'.format(test_df.shape))
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   poem_id     217 non-null    int64 
 1   poem_link   217 non-null    object
 2   poem_style  217 non-null    object
 3   poem_text   217 non-null    object
 4   poem_title  217 non-null    object
 5   poet_cat    217 non-null    object
dtypes: int64(1), object(5)
memory usage: 10.3+ KB
None


The Shape is (217, 6)




,poem_id,poem_link,poem_style,poem_text,poem_title,poet_cat
0,28111,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,اله يعلم ما حلت من دمها وسفكه مستحلا بعدما حرم...,اللّهُ يعلمُ ما حلَّلتَ من دمِها,العصر العباسي
1,28112,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,وكنا نرجي بعد عيسي محمدا لينقذنا من لاعج الضر ...,وكنّا نرجّي بعدَ عيسى محمداً,العصر العباسي
2,28113,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,اشكو الي اله حماتي فما يعلم ما لاقيت منها سواه...,أَشكو إِلى اللّهِ حَماتي فما,العصر العباسي
3,28114,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,قالوا الموفق شيعي فقلت لهم هذا خلاف لناس منه ظ...,قالوا الموفقُ شِيعِيٌّ فقلتُ لهم,العصر العباسي
4,28115,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,ابو الفضل وابن الفضل انت وتربه فغير بديع ان ي...,أبو الفضلِ وابنُ الفضلِ أنتَ وتربهُ,العصر العباسي


from the data view above we notice some problem that need to have action all of them on <br><br>
train_df:
- delete poem_link and poet_link
- we have some missing values in poem_text and poem_title that need to delete
- the poem_text and poem_title have symbol and we want to delete them

test_df:
- we will delete the poem_link



# EDA 
to speed the process we used Pandas Profiling to help us explore our data

In [8]:
report = ProfileReport(train_df)
report

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  3.01it/s]


In [9]:
report = ProfileReport(test_df)
report

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  7.06it/s]


The below code is not work for me but it's for EDA for test and train datasets

In [10]:


# import tensorflow_data_validation as tfdv

# train_stats = tfdv.generate_statistics_from_dataframe(train_df)
# eval_stats = tfdv.generate_statistics_from_dataframe(test_df)
# tfdv.visualize_statistics(
#   lhs_statistics = eval_stats,
#   rhs_statistics = train_stats,
#   lhs_name = 'Eval_Dataset',
#   rhs_name= 'Train_Dataset'
# )

# Cleaning

Now will try to remove the things Mentioned Earlier about the missing data and the Duplicated values in the texts 


In [104]:
test_df.head()

,poem_id,poem_link,poem_style,poem_text,poem_title,poet_cat
0,28111,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,اله يعلم ما حلت من دمها وسفكه مستحلا بعدما حرم...,اللّهُ يعلمُ ما حلَّلتَ من دمِها,العصر العباسي
1,28112,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,وكنا نرجي بعد عيسي محمدا لينقذنا من لاعج الضر ...,وكنّا نرجّي بعدَ عيسى محمداً,العصر العباسي
2,28113,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,اشكو الي اله حماتي فما يعلم ما لاقيت منها سواه...,أَشكو إِلى اللّهِ حَماتي فما,العصر العباسي
3,28114,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,قالوا الموفق شيعي فقلت لهم هذا خلاف لناس منه ظ...,قالوا الموفقُ شِيعِيٌّ فقلتُ لهم,العصر العباسي
4,28115,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,ابو الفضل وابن الفضل انت وتربه فغير بديع ان ي...,أبو الفضلِ وابنُ الفضلِ أنتَ وتربهُ,العصر العباسي


#### Dropping irrelevant columns

In [105]:
train_df = train_df.drop(['poem_link', 'poet_link'], axis=1)
test_df = test_df.drop('poem_link', axis=1)

In [106]:
print('Train Shape: {} \nTest Shape: {}'.format(train_df.shape, test_df.shape))
#Expected: 
#   Train Shape: (57812, 7) 
#   Test Shape: (217, 5)

Train Shape: (57812, 7) 
Test Shape: (217, 5)


#### Dropping the duplicate rows
since there are no duplicate rows were found, after analysis we found some text were written more than once so these values will be reoved since it's could count as duplicate rows in this Situation.

In [107]:
duplicated_train = train_df[train_df.duplicated('poem_text')]
duplicated_train

,poem_id,poem_style,poem_text,poem_title,poet_cat,poet_id,poet_name
614,68900,فصحى,ليس في وسعك يا سيدتي ان تصلحيني فلقد فات القطا...,تصميم,سوريا,7,نزار قباني
1313,67926,فصحى,تلك المنازل كيف حال مقيمها انا قنعنا بعدها برس...,تلك المنازل,لبنان,31,إيليا أبو ماضي
1700,66932,فصحى,ذلك الرجل الذي استطاع اقتحام ملكتي علي صهوة رس...,أشهد برجل على صهوة رسالة,سوريا,35,غادة السمان
2152,5964,فصحى,ارجوان له كتف لا يخالجه الخوف ولبندقية كعب يكت...,الأرجوان,البحرين,68,قاسم حداد
2802,63959,فصحى,ما اسرع ما غادرت حداءق العب لابيع السجاءر ما ا...,تكوينات (1),العراق,92,عدنان الصائغ
...,...,...,...,...,...,...,...
56173,84480,فصحى,يقولون لي الفاظ هجوك عندنا الي القلب من الفاظ ...,يقولون لي ألفاظُ هجوك عندنا,عمان,676,أبزون العماني
57738,86600,فصحى,ابا هاشم هشمتني الشفار فله صبري لذاك الاوار ذك...,أَبا هاشِمٍ هَشمتني الشِّفارُ,العصر الأندلسي,750,المعتمد بن عباد
57747,86621,فصحى,انفحة الروض رقت في صبا السحر من بعد ما بات وال...,أَنفحة الروض رقت في صبا السحرِ,العصر الأندلسي,750,المعتمد بن عباد
57750,86624,فصحى,لولا عيون من الواشين ترمقني وما احاذره من قول ...,لولا عيون من الواشين ترمقني,العصر الأندلسي,750,المعتمد بن عباد


From the result above it's shows that there are 328 record were counted as duplicated and they will be removed.

In [108]:
print('Train Shape before removing: {}'.format(train_df.shape))
train_df = train_df.drop_duplicates('poem_text')
print('Train Shape after removing: {} '.format(train_df.shape))


Train Shape before removing: (57812, 7)
Train Shape after removing: (57484, 7) 


#### Dropping the missing or null values.

In [109]:
print(train_df.isnull().sum())

poem_id       0
poem_style    0
poem_text     1
poem_title    2
poet_cat      0
poet_id       0
poet_name     0
dtype: int64


In [110]:
train_df = train_df.dropna()    # Dropping the missing values.
print(train_df.isnull().sum())   # After dropping the values

poem_id       0
poem_style    0
poem_text     0
poem_title    0
poet_cat      0
poet_id       0
poet_name     0
dtype: int64


Since the values of poem_style in the test all of them are فصحى so we will remove any thing else

In [111]:
train_df = train_df[train_df.poem_style == 'فصحى']

and since the values of the poem_text in the testing dataframe the min start from 51 so, any poem_text in the training dataframe less than 45 will be removed

In [112]:
train_df.poem_text.str.len()

0        2630
1        3343
2        1397
3         799
4         739
         ... 
57807     483
57808     251
57809     384
57810     187
57811     238
Name: poem_text, Length: 57354, dtype: int64

In [113]:
print('Train Shape before removing: {}'.format(train_df.shape))
train_df = train_df[train_df.poem_text.str.len() > 45]
print('Train Shape after removing: {}'.format(train_df.shape))


Train Shape before removing: (57354, 7)
Train Shape after removing: (56776, 7)


over 600 record were removed because the number of charecter in the dataset was less than 45 charecters



#### Functions

In [114]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' # define arabic punctuations

def clean_text(text):
  '''
  DESCRIPTION:
  This function to clean text 
  INPUT: 
  text: string
  OUTPUT: 
  text: string after clean it
  ''' 
  text = re.sub("[a-zA-Z]", "", text) # remove english letters
  text = re.sub('\n', '', text) # remove \n from text
  text = re.sub(r"\s+", ' ',text) #remove long spaces 
  text = re.sub(r'\d+', ' ', text) #remove number
  text = re.sub(r'http\S+', '', text) # remove links
  text = text.translate(str.maketrans('','', arabic_punctuations)) # remove punctuation
  text = ' '.join([word for word in text.split() if word not in stopwords.words("arabic")]) # remove stop word
  text = re.sub(' +', ' ',text) # remove extra space
  text = text.strip() #remove whitespaces
  text = araby.strip_diacritics(text=text) # remove diacritic

  return text

def normalize(text):
  '''
  DESCRIPTION:
  This function to normlaize text 
  INPUT: 
  text: string
  OUTPUT: 
  text: string after normalize
  '''
  text = re.sub("[إأآا]", "ا", text)
  text = re.sub("ى", "ي", text)
  text = re.sub("ؤ", "ء", text)
  text = re.sub("ئ", "ء", text)
  text = re.sub("ة", "ه", text)
  text = re.sub("گ", "ك", text)
    
  return text

In [132]:
train_df.head()

,poem_id,poem_style,poem_text,poem_title,poet_id,poet_name,poet_cat_أفغانستان,poet_cat_إيران,poet_cat_الأردن,poet_cat_الإمارات,...,poet_cat_تونس,poet_cat_سوريا,poet_cat_شعراء العراق والشام,poet_cat_عمان,poet_cat_فلسطين,poet_cat_قطر,poet_cat_لبنان,poet_cat_ليبيا,poet_cat_مصر,poet_cat_موريتانيا
0,21,فصحى,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,أنشودة المطر,2,بدر شاكر السياب,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65546,فصحى,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,أقداح و أحلام,2,بدر شاكر السياب,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,65561,فصحى,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة و...,هوى واحد !,2,بدر شاكر السياب,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66170,فصحى,لا تزيديه لوعة فهو يلقاك لينسي لديك بعض اكتءا...,لا تزيديه لوعة,2,بدر شاكر السياب,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66171,فصحى,عطرت احلامي بهذا الشذي من شعرك المسترسل الاسو...,عبير,2,بدر شاكر السياب,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
test_df.head()

,poem_id,poem_style,poem_text,poem_title,poet_cat_البحرين,poet_cat_السعودية,poet_cat_العراق,poet_cat_العصر الأندلسي,poet_cat_العصر العباسي,poet_cat_سوريا,poet_cat_لبنان,poet_cat_مصر
0,28111,فصحى,اله يعلم ما حلت من دمها وسفكه مستحلا بعدما حرم...,اللّهُ يعلمُ ما حلَّلتَ من دمِها,0,0,0,0,1,0,0,0
1,28112,فصحى,وكنا نرجي بعد عيسي محمدا لينقذنا من لاعج الضر ...,وكنّا نرجّي بعدَ عيسى محمداً,0,0,0,0,1,0,0,0
2,28113,فصحى,اشكو الي اله حماتي فما يعلم ما لاقيت منها سواه...,أَشكو إِلى اللّهِ حَماتي فما,0,0,0,0,1,0,0,0
3,28114,فصحى,قالوا الموفق شيعي فقلت لهم هذا خلاف لناس منه ظ...,قالوا الموفقُ شِيعِيٌّ فقلتُ لهم,0,0,0,0,1,0,0,0
4,28115,فصحى,ابو الفضل وابن الفضل انت وتربه فغير بديع ان ي...,أبو الفضلِ وابنُ الفضلِ أنتَ وتربهُ,0,0,0,0,1,0,0,0


replacing the values of poet_cat to numbercal Representation

In [131]:
train_df = pd.concat([train_df,pd.get_dummies(train_df['poet_cat'], prefix='poet_cat')],axis=1)
train_df.drop(['poet_cat'],axis=1, inplace=True)

In [134]:
test_df = pd.concat([test_df,pd.get_dummies(test_df['poet_cat'], prefix='poet_cat')],axis=1)
test_df.drop(['poet_cat'],axis=1, inplace=True)

checks the values in poem_style columns in both training and testing

In [133]:
print(test_df.poem_style.value_counts())
train_df.poem_style.value_counts()


فصحى    217
Name: poem_style, dtype: int64


فصحى    56776
Name: poem_style, dtype: int64

since they both contain only one value both of them will be removed

In [136]:
train_df.drop(['poem_style'], axis=1, inplace = True)
test_df.drop(['poem_style'], axis=1, inplace = True)

In [137]:
train_df

,poem_id,poem_text,poem_title,poet_id,poet_name,poet_cat_أفغانستان,poet_cat_إيران,poet_cat_الأردن,poet_cat_الإمارات,poet_cat_البحرين,...,poet_cat_تونس,poet_cat_سوريا,poet_cat_شعراء العراق والشام,poet_cat_عمان,poet_cat_فلسطين,poet_cat_قطر,poet_cat_لبنان,poet_cat_ليبيا,poet_cat_مصر,poet_cat_موريتانيا
0,21,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,أنشودة المطر,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65546,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,أقداح و أحلام,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,65561,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة و...,هوى واحد !,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66170,لا تزيديه لوعة فهو يلقاك لينسي لديك بعض اكتءا...,لا تزيديه لوعة,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66171,عطرت احلامي بهذا الشذي من شعرك المسترسل الاسو...,عبير,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57807,86613,لروح صهيل لا تحويه الاوقات ذنبك انك تمتد علي ا...,الوقت,755,عز الدين اليوسف,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
57808,86614,اه لو اني ابني الشمس بعيني من طين هواء وسراب م...,الفلاح,755,عز الدين اليوسف,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
57809,86615,في عينيك يا امي لماذا الدمع منتظم كعقد الءلء ا...,أمّي,755,عز الدين اليوسف,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
57810,86616,النوم يوقظ طرفي الظامي علي لحن تموج من بعيد ار...,عن اللحن,755,عز الدين اليوسف,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [138]:
test_df

,poem_id,poem_text,poem_title,poet_cat_البحرين,poet_cat_السعودية,poet_cat_العراق,poet_cat_العصر الأندلسي,poet_cat_العصر العباسي,poet_cat_سوريا,poet_cat_لبنان,poet_cat_مصر
0,28111,اله يعلم ما حلت من دمها وسفكه مستحلا بعدما حرم...,اللّهُ يعلمُ ما حلَّلتَ من دمِها,0,0,0,0,1,0,0,0
1,28112,وكنا نرجي بعد عيسي محمدا لينقذنا من لاعج الضر ...,وكنّا نرجّي بعدَ عيسى محمداً,0,0,0,0,1,0,0,0
2,28113,اشكو الي اله حماتي فما يعلم ما لاقيت منها سواه...,أَشكو إِلى اللّهِ حَماتي فما,0,0,0,0,1,0,0,0
3,28114,قالوا الموفق شيعي فقلت لهم هذا خلاف لناس منه ظ...,قالوا الموفقُ شِيعِيٌّ فقلتُ لهم,0,0,0,0,1,0,0,0
4,28115,ابو الفضل وابن الفضل انت وتربه فغير بديع ان ي...,أبو الفضلِ وابنُ الفضلِ أنتَ وتربهُ,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
212,11344,اقول باني واحد بوجودي واني كثير في الوجود بجود...,أقول بأني واحد بوجودي,0,0,0,0,1,0,0,0
213,11345,وقال ايضايدل الجزء من مضمون كوني علي ما دل كلي...,وقـال أيضـاً:يدل الجزؤ من مضمون كوني,0,0,0,0,1,0,0,0
214,11346,اذا ما نعت الحق يوما فقيد ولا تطلقن النعت ان ك...,إذا ما نعتَّ الحقَّ يوماً فقيدِ,0,0,0,0,1,0,0,0
215,11347,اني رايت وما رايت وجودي ورايته ذخري ليوم شهودي...,إني رأيتُ وما رأيتُ وجودي,0,0,0,0,1,0,0,0


In [139]:
# apply the above functions to train_df columns
train_df.poem_text = train_df.poem_text.apply(lambda x:clean_text(x))
train_df.poem_text = train_df.poem_text.apply(lambda x:normalize(x))

train_df.poem_title = train_df.poem_title.apply(lambda x:clean_text(x))
train_df.poem_title = train_df.poem_title.apply(lambda x:normalize(x))

# apply the above functions to test_df columns
test_df.poem_text = test_df.poem_text.apply(lambda x:clean_text(x))
test_df.poem_text = test_df.poem_text.apply(lambda x:normalize(x))

test_df.poem_title = test_df.poem_title.apply(lambda x:clean_text(x))
test_df.poem_title = test_df.poem_title.apply(lambda x:normalize(x))

In [140]:
train_df.head()

,poem_id,poem_text,poem_title,poet_id,poet_name,poet_cat_أفغانستان,poet_cat_إيران,poet_cat_الأردن,poet_cat_الإمارات,poet_cat_البحرين,...,poet_cat_تونس,poet_cat_سوريا,poet_cat_شعراء العراق والشام,poet_cat_عمان,poet_cat_فلسطين,poet_cat_قطر,poet_cat_لبنان,poet_cat_ليبيا,poet_cat_مصر,poet_cat_موريتانيا
0,21,عيناك غابتا نخيل ساعه السحر او شرفتان يناي عنه...,انشوده المطر,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65546,انا ازال يدي قدحي ياليل اين تفرق الشرب زلت اشر...,اقداح احلام,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,65561,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبه و...,هوي,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66170,تزيديه لوعه فهو يلقاك لينسي لديك اكتءابه قربي ...,تزيديه لوعه,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66171,عطرت احلامي بهذا الشذي شعرك المسترسل الاسود ال...,عبير,2,بدر شاكر السياب,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
test_df.head()

,poem_id,poem_text,poem_title,poet_cat_البحرين,poet_cat_السعودية,poet_cat_العراق,poet_cat_العصر الأندلسي,poet_cat_العصر العباسي,poet_cat_سوريا,poet_cat_لبنان,poet_cat_مصر
0,28111,اله يعلم حلت دمها وسفكه مستحلا بعدما حرما رايت...,الله يعلم حللت دمها,0,0,0,0,1,0,0,0
1,28112,وكنا نرجي عيسي محمدا لينقذنا لاعج الضر والبلوي...,وكنا نرجي بعد عيسي محمدا,0,0,0,0,1,0,0,0
2,28113,اشكو الي اله حماتي فما يعلم لاقيت سواه عجوز سو...,اشكو الي الله حماتي فما,0,0,0,0,1,0,0,0
3,28114,قالوا الموفق شيعي فقلت خلاف لناس ظهر وكيف يصبح...,قالوا الموفق شيعي فقلت,0,0,0,0,1,0,0,0
4,28115,ابو الفضل وابن الفضل انت وتربه فغير بديع ان يك...,الفضل وابن الفضل انت وتربه,0,0,0,0,1,0,0,0
